# CNN Implementation in TreNet

### Authors: Nathan Ng

### CNN Implementation

- CNN of *k* filters of window size *w* is applied to raw time series data
- ReLU activation function is applied on CNN outputs
- Results are pooled in max pooling layer 
- Finally, pooled outputs are concatenated to form fully connected layer 

- Note: Dropout layer is skipped because optimal TreNet hyperparameters found for NYSE data had dropout of 0.0

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Use GPU if available for PyTorch
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = torch.device(dev)

## Data Pre-Processing

In [3]:
"""
Scales data using Sklearn's MinMaxScaler
Generalized to accept tensors 
"""
class Scaler():
    def __init__(self):
        self.scaler = MinMaxScaler()
        
    def fit_transform(self, data):
        # Check if data is tensor
        if type(data) == torch.Tensor:
            data = torch.Tensor.cpu(data).detach().numpy()
            
        # Check if data is dataframe 
        if type(data) == pd.DataFrame or type(data) == pd.Series:
            data = data.values
            
        # Transform data 
        if len(data.shape) == 1: 
            scaled_data = self.scaler.fit_transform(data.reshape(-1, 1)).flatten()
        else: 
            scaled_data = self.scaler.fit_transform(data)
        
        # Return tensor of scaled data
        return torch.tensor(scaled_data, dtype=torch.float)
    
    def inverse_transform(self, data): 
        # Check if data is tensor
        if type(data) == torch.Tensor:
            data = torch.Tensor.cpu(data).detach().numpy()
            
        # Check if data is dataframe 
        if type(data) == pd.DataFrame or type(data) == pd.Series:
            data = data.values
        
        inverse_data = self.scaler.inverse_transform(data)
        
        # Return tensor of inverse data
        return torch.tensor(inverse_data, dtype=torch.float)

In [4]:
"""
Extracts m sequential data to use to predict n next data 
"""
def extract_data(data, num_input, num_output):
    num_rows = data.shape[0] - num_input - num_output
    input_data = torch.zeros(num_rows, num_input)
    output_data = torch.zeros(num_rows, num_output)
    
    for i in range(num_rows):
        input_data[i] = (data[i:i+num_input])
        output_data[i] = (data[i+num_input:i+num_input+num_output])
    return input_data, output_data

In [5]:
"""
Separates data into train, validation, and test sets
props: (train, valid)
"""
def train_valid_test_split(X, y, props=None):
    if not props: 
        props = (0.5, 0.25)
    elif len(props) != 3: 
        print("Wrong number of parameters")
        return None
    
    train_size = int(X.shape[0] * props[0])
    valid_size = int(X.shape[0] * props[1])
    
    X_train = X[:train_size].to(device)
    y_train = y[:train_size].to(device)
    
    X_valid = X[train_size: (train_size + valid_size)].to(device)
    y_valid = y[train_size: (train_size + valid_size)].to(device)
    
    X_test = X[(train_size + valid_size):].to(device)
    y_test = y[(train_size + valid_size):].to(device)
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

## TreNet CNN Model

In [6]:
class TreNetCNN(nn.Module):
    def __init__(self, num_data, layers=None, num_filters=None, dropout=None, conv_size=3, pooling_size=3, output_size=2):
        """
        layers (int): Number of cnn stacks to create 
        num_filters (list(int)): Number of CNN filters corresponding to same index stacks
        dropout (list(float)): Probability of dropout corresponing to same index stack
        convsize (int, list(int)): Size of filter sizes 
        """
        super(TreNetCNN, self).__init__()
        self.num_data = num_data
        self.layers = layers
        self.num_filters = num_filters
        self.dropout = dropout
        self.conv_size = conv_size
        self.pooling_size = pooling_size
        self.output_size = output_size
        self.cnn_stack = self.create_cnn_stack()
    
    def create_cnn_stack(self):
        # Initialize default stack settings
        if not self.layers:
            self.layers = 2
        if not self.num_filters:
            self.num_filters = [128] * self.layers
        if not self.dropout:
            self.dropout = [0.0] * self.layers
        if type(self.conv_size) == int:
            self.conv_size = [self.conv_size] * self.layers
        
        # Create cnn stacks 
        cnn_stacks = []
        num_channels = 1
        updated_data = self.num_data
        for i in range(self.layers):
            cnn_stack = nn.Sequential(
                nn.Conv1d(in_channels=num_channels, out_channels=self.num_filters[i], kernel_size=self.conv_size[i]),
                nn.ReLU(),
                nn.MaxPool1d(kernel_size=self.pooling_size, stride=1),
                nn.Dropout(p=self.dropout[i])
            )
            num_channels = self.num_filters[i]
            cnn_stacks.append(cnn_stack)
            
            # Keep track of current size of data 
            updated_data = updated_data - self.conv_size[i] + 1 - self.pooling_size + 1
            new_data_size = self.num_filters[i] * updated_data
            
        # Add fully connected layer at end to output trend duration and slope
        output_layer = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(new_data_size, self.output_size)
        )
        cnn_stacks.append(output_layer)
            
        # Combine cnn stacks 
        return nn.Sequential(*cnn_stacks)
    
    def forward(self, x):
        x = torch.reshape(x, (x.shape[0], 1, -1))
        output = self.cnn_stack(x)
        return output

## TreNet CNN Training and Loss Function

In [7]:
"""
Create training loop
"""
def train_loop(n_epochs, X, y, model, loss_fn, optimizer, printout=False):
    for i in range(n_epochs):
        # Compute prediction and loss 
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print loss per epoch
        if printout and i % 100 == 0:
            print(f"Epoch {i}:\n--------------")
            print(f"Train Loss: {np.sqrt(loss.item())}")
            print()
            
    # Print final loss after training 
    if printout:
        print(f"Final:\n--------------")
        print(f"Train Loss: {np.sqrt(loss.item())}")
        print()

## Test CNN Stack Model on Raw Stock Prices

In [8]:
# Load in NYSE data
df = pd.read_csv("../data/raw/indexProcessed.csv")
NYSE_df = df[df["Index"] == "NYA"].loc[:, "Open"].reset_index(drop=True)

# Create subset of data 
NYSE_sub = NYSE_df[:1000]

In [9]:
# Scale data
scaler = Scaler()
sub = scaler.fit_transform(NYSE_sub).to(device)

In [10]:
# Extract samples and create train test sets
X, y = extract_data(sub, 99, 1)
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(X, y)

In [11]:
# Set parameters for model and training 
cnn_params = {
    'num_data': 99,
    'layers': 2, 
    'num_filters': [32, 32],
    'dropout': [0.1, 0.1], 
    'conv_size': [2, 4], 
    'output_size': 1
}

learning_rate = 0.0001
epochs = 1000

In [12]:
# Initialize model, loss function, and optimizer
model = TreNetCNN(**cnn_params).to(device)

# Initialize loss function 
loss_fn = nn.MSELoss()

# Initialize optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Train model
train_loop(epochs, X_train, y_train, model, loss_fn, optimizer, printout=True)

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0:
--------------
Train Loss: 0.5814115310096946

Epoch 100:
--------------
Train Loss: 0.09614612752969987

Epoch 200:
--------------
Train Loss: 0.07200849706935462

Epoch 300:
--------------
Train Loss: 0.067727111159626

Epoch 400:
--------------
Train Loss: 0.06122294528802654

Epoch 500:
--------------
Train Loss: 0.057402187414044036

Epoch 600:
--------------
Train Loss: 0.05382409746469356

Epoch 700:
--------------
Train Loss: 0.052727071821517056

Epoch 800:
--------------
Train Loss: 0.05229615994509662

Epoch 900:
--------------
Train Loss: 0.043817343089010384

Final:
--------------
Train Loss: 0.0448866011163473



In [14]:
# Compare predictions and actual values 
inverse_test_y = scaler.inverse_transform(y_test)

pred_test_y = model(X_test)
inverse_pred_test_y = scaler.inverse_transform(pred_test_y)

In [15]:
print("Test Loss: " + str(loss_fn(inverse_pred_test_y, inverse_test_y)**(1/2)))

print("Predicted values: \n" + str(inverse_test_y[:10]))

print("Actual values: \n" + str(inverse_pred_test_y[:10]))

Test Loss: tensor(21.2793)
Predicted values: 
tensor([[586.9500],
        [589.1700],
        [587.5800],
        [583.4600],
        [580.8200],
        [582.1900],
        [583.8800],
        [588.1100],
        [591.9200],
        [590.7500]])
Actual values: 
tensor([[617.7910],
        [606.2527],
        [609.2711],
        [608.5897],
        [587.4570],
        [559.3215],
        [605.8821],
        [608.0587],
        [588.2817],
        [587.3508]])
